<a href="https://colab.research.google.com/github/ivan-rib/Amablindados/blob/main/analise_dados_ama_ivan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 34.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=1764bf4828bec4e2d1e50d84dab7c6315cc0398151e8a86b99f35617e6cc62fe
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from google.colab import drive
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [127]:
#lendo csv do primeiro dataset
path = ['/content/drive/MyDrive/arquivos_ama/OneDrive_1_25-05-2022-20220602T184014Z-001.zip (Unzipped Files)/OneDrive_1_25-05-2022/01S2020.csv',
        '/content/drive/MyDrive/arquivos_ama/OneDrive_1_25-05-2022-20220602T184014Z-001.zip (Unzipped Files)/OneDrive_1_25-05-2022/1S2017.csv',
        '/content/drive/MyDrive/arquivos_ama/OneDrive_1_25-05-2022-20220602T184014Z-001.zip (Unzipped Files)/OneDrive_1_25-05-2022/1S2018.csv',
        '/content/drive/MyDrive/arquivos_ama/OneDrive_1_25-05-2022-20220602T184014Z-001.zip (Unzipped Files)/OneDrive_1_25-05-2022/1S2019.csv',
        '/content/drive/MyDrive/arquivos_ama/OneDrive_1_25-05-2022-20220602T184014Z-001.zip (Unzipped Files)/OneDrive_1_25-05-2022/1S2021.csv',
        '/content/drive/MyDrive/arquivos_ama/OneDrive_1_25-05-2022-20220602T184014Z-001.zip (Unzipped Files)/OneDrive_1_25-05-2022/2S2017.csv',
        '/content/drive/MyDrive/arquivos_ama/OneDrive_1_25-05-2022-20220602T184014Z-001.zip (Unzipped Files)/OneDrive_1_25-05-2022/2S2018.csv',
        '/content/drive/MyDrive/arquivos_ama/OneDrive_1_25-05-2022-20220602T184014Z-001.zip (Unzipped Files)/OneDrive_1_25-05-2022/2S2019.csv',
        '/content/drive/MyDrive/arquivos_ama/OneDrive_1_25-05-2022-20220602T184014Z-001.zip (Unzipped Files)/OneDrive_1_25-05-2022/2S2021.csv']



In [138]:
csv= spark.read.format('csv')\
  .option("header","true")\
	.option("delimiter","\t")\
  .option("encoding", "utf-16")\
  .option("multiline", "true")\
  .load('/content/drive/MyDrive/arquivos_ama/OneDrive_1_25-05-2022-20220602T184014Z-001.zip (Unzipped Files)/OneDrive_1_25-05-2022/*.csv')
	
  

In [136]:
csv.show(5)

+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
|        CNPJ emit.|            Emitente|   No.|Data Emissão|Valor NF|Data Registro|Créditos|Situação do Crédito|
+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
|
|
|
|
|
+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
only showing top 5 rows



In [140]:
csv.count()

165464

In [139]:
csv.printSchema()

root
 |-- CNPJ emit.: string (nullable = true)
 |-- Emitente: string (nullable = true)
 |-- No.: string (nullable = true)
 |-- Data Emissão: string (nullable = true)
 |-- Valor NF: string (nullable = true)
 |-- Data Registro: string (nullable = true)
 |-- Créditos: string (nullable = true)
 |-- Situação do Crédito: string (nullable = true)



In [144]:
data = csv.withColumnRenamed('CNPJ emit.','cnp_emit') \
          .withColumnRenamed('Emitente','emitente')\
          .withColumnRenamed('No.','numero') \
          .withColumnRenamed('Data Emissão','data_emissao')\
          .withColumnRenamed('Valor NF','valor_nf')\
          .withColumnRenamed('Data Registro','data_registro')\
          .withColumnRenamed('Créditos','creditos')\
          .withColumnRenamed('Situação do Crédito','situacao_do_credito')

In [145]:
data = data.withColumn('valor_nf', regexp_replace('valor_nf', ',', '.'))\
           .withColumn('creditos', regexp_replace('creditos', ',', '.'))

In [146]:
data.toPandas()


,cnp_emit,emitente,numero,data_emissao,valor_nf,data_registro,creditos,situacao_do_credito
0,05.234.750/0001-41,TX BURGER LTDA,77882,01/01/2020,78.8,08/01/2020,0,A Calcular\r
1,47.508.411/1476-88,CIA BRASILEIRA DE DISTRIBUICAO,20087,01/01/2020,22.71,01/01/2020,0,A Calcular\r
2,09.060.964/0108-39,PIMENTA VERDE ALIMENTOS LTDA,82366,01/01/2020,35.9,08/01/2020,0,A Calcular\r
3,61.412.110/0307-39,DROGARIA SAO PAULO S A,234178,01/01/2020,21.77,08/01/2020,0,A Calcular\r
4,46.952.032/0001-98,PANIFICADORA E CONFEITARIA QUELUZ LTDA,3305,01/01/2020,80.95,08/01/2020,0,A Calcular\r
...,...,...,...,...,...,...,...,...
165459,61.116.828/0001-02,CALVO COMERCIO E IMPORTACAO LTDA,411588,02/01/2018,360.82,02/01/2018,0,Liberado\r
165460,67.616.128/0001-55,MERCADINHO AYUMI LTDA,43860,02/01/2018,194.02,02/01/2018,45,Liberado\r
165461,54.651.716/0011-50,SUPRICORP SUPRIMENTOS LTDA,3100306,02/01/2018,81.87,02/01/2018,0,Liberado\r
165462,67.616.128/0001-55,MERCADINHO AYUMI LTDA - 1,27754,02/01/2018,194.02,02/01/2018,0,Liberado\r


In [123]:
spark.sql("select * from data_data").show(5)


+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
|          cnp_emit|            emitente|numero|data_emissao|valor_nf|data_registro|creditos|situacao_do_credito|
+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
|
|
|
|
|
+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
only showing top 5 rows

